In [2]:
!pip install faker

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 4, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.1 MB/s eta 0:00:00a 0:00:010m


In [3]:
import requests
from faker import Faker
import pandas as pd
import random
from datetime import datetime, timedelta
import random
from pyspark.sql import SparkSession
from pyspark.sql import Row
from datetime import date, timedelta

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 5, Finished, Available, Finished)

In [4]:
# Initialiser Faker
fake = Faker('fr_FR')

# Country code list
country_code = ['CI', 'SN', 'ML', 'TG', 'BJ', 'BF', 'GN', 'NE', 'CM', 'GA']
# Devise for each country
devise = ['XOF', 'XOF', 'XOF', 'XOF', 'XOF', 'XOF', 'GNF', 'XOF', 'XAF', 'XAF']

def generate_client(id_clt):
    return {
        "id_clt": id_clt,  # identifiant du client
        "num_compt": fake.bban(),  # Numéro de compte bancaire fictif
        "nom": fake.last_name(),  # Nom du client
        "prenom": fake.first_name(),  # Prénom du client
        "tel": fake.phone_number(),  # Numéro de téléphone
        "code_pays": random.choices(country_code)[0],  # pays d'origine du client
        "date_inscription": fake.date_between(start_date='-5y', end_date='today').isoformat(),  # date d'inscription
        "agence_id_inscription": random.randint(1, 15),  # ID agence
        "type_personne": random.choice(["physique", "morale"]),  # type de client
        "score_kyc": random.randint(0, 100),  # score KYC
        "nombre_produit_souscrit": random.randint(1, 5),  # produits souscrits
        "type_compt": random.choice(["premium-plus", "premium", "normal"])  # type de compte
    }

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 6, Finished, Available, Finished)

In [5]:
spark = SparkSession.builder \
    .appName("CreateDataFrame") \
    .getOrCreate()


StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 7, Finished, Available, Finished)

In [6]:
clients = [generate_client(i) for i in range(1, 2001)]

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 8, Finished, Available, Finished)

In [7]:
df = spark.createDataFrame([Row(**client) for client in clients])

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 9, Finished, Available, Finished)

In [7]:
display(df)

StatementMeta(, 94c9296f-ba0f-4e1a-b43d-45bcae6eb86d, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 089e6c99-e646-4b2d-9b97-6be6f7dc869c)

In [8]:
type_trx_list = ['TRANSFERT', 'DEPOT', 'RETRAIT']
status_trx_list = ['SUCCES', 'ECHEC']
bank_devices =  ['MOBILE', 'ATM', 'AGENCE', 'WEB']
bban_list = df.select('num_compt').rdd.flatMap(lambda x: x).collect()
dict_devises =  dict(zip(country_code, devise))
number_agence = 10
limit_table_core_banking =  2000

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 10, Finished, Available, Finished)

In [9]:
def generate_transactions_between(type_transaction, list_bban =  bban_list):
    if type_transaction == 'TRANSFERT':
        num_compt_init = random.choice(bban_list)
        num_compt_dest = random.choice(bban_list)
    elif type_transaction == 'DEPOT':
        compte = random.choice(bban_list)
        num_compt_init = compte
        num_compt_dest = compte
    elif type_transaction == 'RETRAIT':
        num_compt_init = None
        num_compt_dest = random.choice(bban_list)
    else:
        raise ValueError("Transaction Type not found")

    return {'init': num_compt_init, 'dest':num_compt_dest}


# Generate 500 transactions
def generate_transaction(id):

    date_trx = fake.date_time_between(start_date='-2y', end_date='now')
    type_trx = random.choice(type_trx_list)
    between = generate_transactions_between(type_transaction = type_trx, list_bban = bban_list)
    montant_trx = round(random.uniform(1000, 500000), 2)
    num_compt_init = between['init']
    num_compt_dest = between['dest']
    agence_id = random.randint(1, number_agence)
    agence_nom = f"AGENCE-XX{agence_id}"
    status_trx = random.choice(status_trx_list)
    code_pays = random.choice(country_code)
    devise = dict_devises[f'{code_pays}']
    device_of_trx = random.choice(bank_devices)

    return {
        'id_trx': f"TRX-{id}",
        'date_trx': date_trx.strftime('%Y-%m-%d %H:%M:%S'),
        'type_trx': type_trx,
        'montant_trx': montant_trx,
        'num_compt_init': num_compt_init,
        'num_compt_dest': num_compt_dest,
        'agence_id': agence_id,
        'agence_nom': agence_nom,
        'status_trx': status_trx,
        'code_pays': code_pays,
        'devise': devise,
        'device_of_trx': device_of_trx
    }

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 11, Finished, Available, Finished)

In [10]:
# Dataset Creation 
data = [generate_transaction(i+1) for i in range(limit_table_core_banking)]
core_banking =  spark.createDataFrame([Row(**trx) for trx in data])

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 12, Finished, Available, Finished)

In [11]:
display(core_banking)

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 57433e57-bf5d-47de-891a-c3c4bc1306e2)

In [12]:
uniques_countries_crm =  df.select('code_pays').distinct().collect()
uniques_countries_crm = [row['code_pays'] for row in uniques_countries_crm ]
uniques_countries_crm

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 14, Finished, Available, Finished)

['CI', 'NE', 'GN', 'CM', 'TG', 'BF', 'BJ', 'SN', 'ML', 'GA']

In [13]:
for country in uniques_countries_crm:
    dt =  df.filter(df.code_pays == f"{country}")
    # create table  associate to country
    dt.write.mode('overwrite').saveAsTable(f'source_crm_{country}')
    print(f'=========== TABLE DELTA - SOURCE CRM --{country}-- CREATED WITH SUCCESS ===========')


StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 15, Finished, Available, Finished)

=========== TABLE DELTA - SOURCE CRM --CI-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --NE-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --GN-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --CM-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --TG-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --BF-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --BJ-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --SN-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --ML-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CRM --GA-- CREATED WITH SUCCESS ===========


In [14]:
unique_type_trx =  core_banking.select('type_trx').distinct().collect()
unique_type_trx =  [row['type_trx'] for row in unique_type_trx]
unique_type_trx

StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 16, Finished, Available, Finished)

['RETRAIT', 'DEPOT', 'TRANSFERT']

In [15]:
for trx in unique_type_trx: 
    dtrx =  core_banking.filter(core_banking.type_trx == f"{trx}")
    # create table to type_trx
    dtrx.write.mode('overwrite').saveAsTable(f'source_core_banking_{trx}')
    print(f'=========== TABLE DELTA - SOURCE CORE BANKING --{trx}-- CREATED WITH SUCCESS ===========')


StatementMeta(, dd199908-322e-4326-b72e-d61b3295d887, 17, Finished, Available, Finished)

=========== TABLE DELTA - SOURCE CORE BANKING --RETRAIT-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CORE BANKING --DEPOT-- CREATED WITH SUCCESS ===========
=========== TABLE DELTA - SOURCE CORE BANKING --TRANSFERT-- CREATED WITH SUCCESS ===========
